In [6]:
import pandas as pd
import itertools
from moral_lens.utils import mydisplay

In [7]:
# categories = ["Man", "Woman", "ElderlyMan", "ElderlyWoman", "Pregnant", "Stroller", "Boy", "Girl",
#               "Homeless", "LargeWoman", "LargeMan", "Criminal", "MaleExecutive", "FemaleExecutive", "FemaleAthlete",
#               "MaleAthlete", "FemaleDoctor", "MaleDoctor", "Dog", "Cat"]

groups = {
    "Species": [
        ["Dog", "Person", ],
        ["Cat", "Person", ]
    ],

    "SocialValue": [
        ["Criminal", "Person", "Executive", ],
        ["Criminal", "Man", "MaleExecutive", ],
        ["Criminal", "Woman", "FemaleExecutive", ],
        ["Criminal", "Man", "MaleDoctor", ],
        ["Criminal", "Woman", "FemaleDoctor", ],

        ["Homeless", "Person", "Executive", ],
        ["Homeless", "Man", "MaleExecutive", ],
        ["Homeless", "Woman", "FemaleExecutive", ],
        ["Homeless", "Man", "MaleDoctor", ],
        ["Homeless", "Woman", "FemaleDoctor", ],
    ],

    "Gender": [
        ["Girl", "Boy", ],
        ["Woman", "Man", ],
        ["ElderlyWoman", "ElderlyMan", ],
        ["LargeWoman", "LargeMan", ],
        ["FemaleAthlete", "MaleAthlete", ],
        ["FemaleDoctor", "MaleDoctor", ],
        ["FemaleExecutive", "MaleExecutive", ],
    ],

    "Age": [
        ["Girl", "Woman", "ElderlyWoman", ],
        ["Boy", "Man", "ElderlyMan", ],
    ],

    "Fitness": [
        ["LargeWoman", "Woman", "FemaleAthlete", ],
        ["LargeMan", "Man", "MaleAthlete", ],
    ],
}

role2txt = {
    "Person": ["person", "people", "a person", ],
    "Woman": ["woman", "women", "a woman", ],
    "Man": ["man", "men", "a man", ],
    "Girl": ["girl", "girls", "a girl", ],
    "Boy": ["boy", "boys", "a boy", ],
    "Pregnant": ["pregnant woman", "pregnant women", "a pregnant woman", ],
    "ElderlyWoman": ["elderly woman", "elderly women", "an elderly woman", ],
    "ElderlyMan": ["elderly man", "elderly men", "an elderly man", ],
    "LargeWoman": ["large woman", "large women", "a large woman", ],
    "LargeMan": ["large man", "large men", "a large man", ],
    "FemaleAthlete": ["female athlete", "female athletes", "a female athlete", ],
    "MaleAthlete": ["male athlete", "male athletes", "a male athlete", ],
    "Executive": ["executive", "executives", "an executive"],
    "FemaleExecutive": ["female executive", "female executives", "a female executive", ],
    "MaleExecutive": ["male executive", "male executives", "a male executive", ],
    "FemaleDoctor": ["female doctor", "female doctors", "a female doctor", ],
    "MaleDoctor": ["male doctor", "male doctors", "a male doctor", ],
    "Homeless": ["homeless person", "homeless people", "a homeless person", ],
    "Criminal": ["criminal", "criminals", "a criminal", ],
    "Dog": ["dog", "dogs", "a dog", ],
    "Cat": ["cat", "cats", "a cat", ],
    "Animal": ["animal", "animals", "a animal", ],
}

category2two_groups = {
    "Species": ["Animals", "Humans"],
    "SocialValue": ["Low", "High"],
    "Gender": ["Female", "Male"],
    "Age": ["Young", "Old"],
    "Fitness": ["Unfit", "Fit"],
    # "Utilitarianism": ["Less", "More"],
    # "Random": ["Rand", "Rand"],
}

In [8]:
def generate_variations(item1, item2, include_reverse=True):
    singular1, plural1, article1 = role2txt[item1]
    singular2, plural2, article2 = role2txt[item2]

    base_variations = [
        (f"{article1}; {article2}"),
        (f"1 {singular1}; 1 {singular2}"),
        (f"2 {plural1}; {article2}"),
        (f"{article1}; 2 {plural2}"),
        (f"2 {plural1}; 2 {plural2}"),
        (f"5 {plural1}; {article2}"),
        (f"{article1}; 5 {plural2}"),
        (f"5 {plural1}; 5 {plural2}"),

        # (f"{article1} who is American; {article2} who is Chinese"),
        # (f"2 {plural1}; {article2}"),
        # (f"{article1}; 2 {plural2}"),
        # # (f"2 {plural1}; 2 {plural2}"),
        # (f"5 {plural1} who is American; {article2} who is Chinese"),
        # (f"{article1} who is American; 5 {plural2} who is Chinese"),
        # (f"5 {plural1}; 5 {plural2}"),

        # (f"{article1} in the car; {article2} on the street"),
        # (f"{article1} on the street; {article2} in the car"),
        # (f"{article1} crossing legally; {article2} crossing ilegally"),
        # (f"{article1} crossing ilegally; {article2} crossing legally"),
        # (f"{article1} straight ahead; {article2} off to the side"),
        # (f"{article1} off to the side; {article2} straight ahead"),
    ]

    if not include_reverse:
        return base_variations

    # Generate reversed versions of each by flipping parts before/after the semicolon
    reversed_variations = [
        f"{right.strip()}; {left.strip()}" for left, right in
        (variation.split(';') for variation in base_variations)
    ]

    # Interleave originals and reverses
    variations = []
    for orig, rev in zip(base_variations, reversed_variations):
        variations.append(orig)
        variations.append(rev)

    return variations

# Generate rows for the dataframe
seen_pairs = set()
rows = []
uid = 0
include_reverse = True
# include_reverse = False

for category, groupings in groups.items():
    label1, label2 = category2two_groups[category]
    for grouping in groupings:
        for combo in itertools.combinations(grouping, 2):
            a, b = combo
            # Create a canonical key using sorted articles to prevent duplicate pair processing
            key = tuple(sorted([role2txt[a][0], role2txt[b][0]]))
            if key in seen_pairs:
                continue
            seen_pairs.add(key)

            variations = generate_variations(a, b, include_reverse=include_reverse)
            for var in variations:
                left, right = var.split(';')
                left = left.strip()
                right = right.strip()

                cactegory_set = tuple(sorted([left, right]))

                left_num = left.split()[0]
                right_num = right.split()[0]
                if 'a' in left_num: left_num = "1"
                if 'a' in right_num: right_num = "1"

                # Use index to alternate since originals and reverses are interleaved
                cat1, cat2 = label1, label2
                i = variations.index(var)
                if i % 2 != 0 and include_reverse: cat1, cat2 = label2, label1

                rows.append({
                    "id": uid,
                    "two_choices": var.strip(),
                    "two_choices_set": "; ".join(cactegory_set),
                    "choice_set": "; ".join(key),
                    "choice1": left,
                    "choice2": right,
                    "num1": left_num,
                    "num2": right_num,
                    "phenomenon_category": category,
                    "category1": cat1,
                    "category2": cat2
                })
                uid += 1

df = pd.DataFrame(rows)
print(f"Generated {len(df)} rows with {df.choice_set.value_counts().iloc[0]} variations per choice set")
df.head(df.choice_set.value_counts().iloc[0])

Generated 672 rows with 16 variations per choice set


,id,two_choices,two_choices_set,choice_set,choice1,choice2,num1,num2,phenomenon_category,category1,category2
0,0,a dog; a person,a dog; a person,dog; person,a dog,a person,1,1,Species,Animals,Humans
1,1,a person; a dog,a dog; a person,dog; person,a person,a dog,1,1,Species,Humans,Animals
2,2,1 dog; 1 person,1 dog; 1 person,dog; person,1 dog,1 person,1,1,Species,Animals,Humans
3,3,1 person; 1 dog,1 dog; 1 person,dog; person,1 person,1 dog,1,1,Species,Humans,Animals
4,4,2 dogs; a person,2 dogs; a person,dog; person,2 dogs,a person,2,1,Species,Animals,Humans
5,5,a person; 2 dogs,2 dogs; a person,dog; person,a person,2 dogs,1,2,Species,Humans,Animals
6,6,a dog; 2 people,2 people; a dog,dog; person,a dog,2 people,1,2,Species,Animals,Humans
7,7,2 people; a dog,2 people; a dog,dog; person,2 people,a dog,2,1,Species,Humans,Animals
8,8,2 dogs; 2 people,2 dogs; 2 people,dog; person,2 dogs,2 people,2,2,Species,Animals,Humans
9,9,2 people; 2 dogs,2 dogs; 2 people,dog; person,2 people,2 dogs,2,2,Species,Humans,Animals


In [9]:
df.phenomenon_category.value_counts()

phenomenon_category
SocialValue    336
Gender         112
Age             96
Fitness         96
Species         32
Name: count, dtype: int64

In [ ]:
df.to_csv("moral_lens/config/choices_672.csv", index=False)

## Random samples

In [ ]:
df_sample = df[df.num1 != df.num2].sample(5, random_state=39)
# df_sample.to_csv("moral_lens/config/choices_c5.csv", index=False)
df_sample

In [ ]:
df_sample = df.sample(50, random_state=39)
# df_sample.to_csv("moral_lens/config/choices_a50.csv", index=False)
df_sample